In [1]:

import sys 
sys.path.append(r'C:\Users\sergiojesus\Desktop\Diogo2017\EQS_LOCAL\Bitbucket_eqs')
import analytics_stat.distributions as di

import warnings
warnings.filterwarnings('ignore')

import datetime
import scipy.stats as s
#import scipy.optimize as opt
#import scipy.special as special
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline
# %matplotlib notebook

import os
import json
import pandas as pd
import qgrid
from IPython.display import HTML, Image, display, clear_output
# display(HTML("<style>.container { width:95% !important; }</style>"))

# py.sign_in('diogo.antunes.goncalves', 'kt7T5JadNn2AhWk3DdPR')

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

display(HTML('''<script>
code_show=false;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To hide/show code, click <a href="javascript:code_toggle()">here</a>.''')) 



In [2]:
import traitlets
from tkinter import Tk, filedialog


class Plotting:
    
    def __init__(self):
        pass
    
    def plot_line(self, json, x, y, name):
        x = json.x
        y = json.y
        name = json.name
        trace_line = go.Scatter(x = x, y = y, mode = 'lines', opacity = 0.8, name = name, xaxis = 'x', yaxis = 'y')
        
    def plot_scatter(self,  x, y, name):
        x = json.x
        y = json.y
        name = json.name
        trace_scatter = go.Scatter(x = x, y = y, mode = 'markers', opacity = 0.8, name = fn, xaxis = 'x', yaxis = 'y')
        
    def plot_histogram(self,  x, name, normed):
        x= json.x
        normed = json.type  # 'probability'
        trace_histogram = go.Histogram(x=x, histnorm=normed)
                           
    def create_layout(self, title, xlabel, ylabel):
        title = json.title
        xlabel = json.xlabel
        ylabel = json.ylabel
        layout = Layout(title=title, yaxis=dict(title=ylabel)) 
        
    def draw(self,trace,layout):
        fig = go.Figure(data = [trace_histogram],layout=layout)
        py.iplot(fig)

        

class StatPlot(Plotting):
    
    def __init__(self):
        
        pass
    
    def pdf(self, json):
        
        pdf_mle = dist_funs[dist_index[0]].pdf(params_dist_mle,  x_plot)



class DisplayCalc:
    
    def __init__(self):
        
        # Select file button
        self._bsf = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bsf.add_traits(files=traitlets.traitlets.List())
        self._bsf.description = "Select Model File"
        self._bsf.icon = "square-o"
        self._bsf.on_click(self.select_json)
        
        
        # Load Json button
        self._blj = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._blj.description = "Load Model"
        self._blj.icon = 'download'
        self._blj.on_click(self.load_json_model)
        self._blj.disabled = True
        
        # Plot button
        self._bplot = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bplot.description = 'Plot'
        self._bplot.icon = "area-chart"
        self._bplot.on_click(self.plot_result)
        self._bplot.disabled = True
        
        # Save model button
        self._bsm = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bsm.description = 'Save Analysis'
        self._bsm.icon = "floppy-o"
        self._bsm.on_click(self.save_model_json)
        self._bsm.disable = True
        
        # Select Plot type
        self._dd_plots = widgets.Dropdown(
            options=['PDF','Unreliability','Reliability','Bx Life',
#                      'ProbPlot',
                     'MTBF'],
            value='PDF',
            description='Select Analysis:',
            disabled=False,
            layout=widgets.Layout(width='300px', height='50px'))
        
        # define layout
#         box_layout = widgets.Layout(display='center',
#                             flex_flow='row',
#                             align_items='center',
#                             border='None',
#                             justify_content='space-between',
#                             width='800px')
        box_layout = widgets.Layout(display='center',
                                    flex_flow='row',
                                    # align_items='center',
                                    border='None',
                                    justify_content='center',
                                    # width='500px'
                                    )
        
        self._box_a = widgets.Box(children=[self._bsf,
                                            self._blj,  
#                                           self._bsm
                                            ], layout=box_layout)
        
        self._box_b = widgets.Box(children=[self._dd_plots,self._bplot,
                                          ], layout=box_layout)
        
        self._result_output = widgets.Output()
        
        
        display(widgets.VBox([self._box_a, self._box_b, self._result_output]))
        
    def select_json(self,b): 
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.',
                  '-topmost', True)
        
        new_file = filedialog.askopenfilename(multiple=True,filetypes = [("JSON",'*.json')])
        if new_file != "":
            self._bsf.files = new_file

        print('Selected file: {}'.format(self._bsf.files ))

        self._bsf.description = "Model Selected"
        self._bsf.icon = "check-square-o"
        self._bsf.button_style = "success"
        
        self._blj.disabled = False

        
        
    def load_json_model(self, b):
        
        file_path = self._bsf.files
        if (type(file_path) == list) and (len(file_path)>0):
            with open(file_path[0], 'r') as f:
                self._json_file = json.load(f)
                
            self._client = self._json_file['client_name']
            self._clientid = self._json_file['client_id']
            self._symptom = self._json_file['symptom']
                
            self._blj.button_style = "success"
            print('JSON loaded:')
            print(self._json_file)
            
            self._bplot.disabled = False
        else:
            print('Select file.')
            
            
    def save_model_json(self, b):
        json_model = dict()
        json_model['version'] = 0.1 
        json_model['saved_time'] = str(datetime.datetime.now())
        
        if self._symptom is None:
            json_model['symptom'] = None
        elif isinstance(self._symptom, int):
            json_model['symptom'] = int(self._symptom)
        elif isinstance(self._symptom, list):
            json_model['symptom'] = self._symptom
        else:
            raise TypeError('Undefined type for Symptom.')
            
        json_model['client_id'] = int(self._clientid)
        json_model['client_name'] = self._client
        
        json_model['input_json'] = os.path.splitext(os.path.basename(d._bsf.files[0]))[0]
        json_model['competing_models'] = list(self._sdist.value)
        
        json_model['best_model'] = dict()
        json_model['best_model']['law'] = self._combs[0].iloc[0].name
        json_model['best_model']['MLE'] = dict()
        json_model['best_model']['MLE']['params'] = self._res_mle[json_model['best_model']['law']]['params']
        json_model['best_model']['MLE']['goodness_of_fit'] = self._res_mle[json_model['best_model']['law']]['goodness_of_fit']
        
        json_model['best_model']['LSQ'] = dict()
        json_model['best_model']['LSQ']['params'] = self._res_ls[json_model['best_model']['law']]['params']
        json_model['best_model']['LSQ']['goodness_of_fit'] = self._res_ls[json_model['best_model']['law']]['goodness_of_fit']
                                
        with open("../models/stat/model_{}_{}.json".format(json_model['input_json'],
                                                           json_model['best_model']['law']), 'w') as f:
            json.dump(json_model, f)
        
        self._bsm.button_style = 'success'
        
        
    def plot_result(self,b):
        
        plot_obj = self.rda_analysis(self._dd_plots.value)
        
        with self._result_output:
            clear_output()
            py.iplot(plot_obj)
            
    def rda_analysis(self, analysis):
        
        # display plotly graphs in Output widgets
        layout_graph = go.Layout(
            # xaxis=dict(
            #     title='Equipment',
            # ),
            xaxis=dict(tickangle=40),
            yaxis=dict(
                title='',
            )
        )        
        
        if analysis=='PDF':
            t_final = 130
            t_range = np.arange(0,t_final)
            pdf_y = di.Weibull().pdf(d._json_file['best_model']['MLE']['params'],
                                     t_range)
            
            layout_graph.title = analysis
            
            return go.Figure(data=[go.Scatter(x=t_range, y=pdf_y,
                                                   marker=dict(
                                                       color='#d04e09',
                                                       line=dict(
                                                           color='#d04e09',
                                                           width=1.5,
                                                       )))], layout=layout_graph)            
        elif analysis=='Unreliability':
            t_final = 130
            t_range = np.arange(0, t_final)
            cdf_y = di.Weibull().cdf(d._json_file['best_model']['MLE']['params'],
                                     t_range)
            
            layout_graph.title = analysis
            layout_graph.yaxis.title = 'F(t)'
            layout_graph.xaxis.title = 'Time (t)'
            
            
            return go.Figure(data=[go.Scatter(x=t_range, y=cdf_y,
                                                   marker=dict(
                                                       color='#d04e09',
                                                       line=dict(
                                                           color='#d04e09',
                                                           width=1.5,
                                                       )))], layout=layout_graph)
            
            
        elif analysis=='Reliability':
            t_final = 130
            t_range = np.arange(0,t_final)
            cdf_y = di.Weibull().cdf(d._json_file['best_model']['MLE']['params'],
                                     t_range)
            
            layout_graph.title = analysis
            layout_graph.yaxis.title = 'R(t)'
            layout_graph.xaxis.title = 'Time (t)'
            
            return go.Figure(data=[go.Scatter(x=t_range, y=1-cdf_y,
                                                   marker=dict(
                                                       color='#d04e09',
                                                       line=dict(
                                                           color='#d04e09',
                                                           width=1.5,
                                                       )))], layout=layout_graph)
            
        elif analysis=='Bx Life':
            perc = np.array([0.01, .1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                  20, 30, 40, 50, 60, 70, 80, 85, 90, 95,
                  99.99
                ])
            
            percentile = di.Weibull().ppf(d._json_file['best_model']['MLE']['params'],perc/100)

            table_of_percentiles = pd.DataFrame({'BX%':perc,'Percentile (t)':percentile})
            
            layout_graph.title = 'B10 Life: {0:0.2f}'.format(table_of_percentiles.loc[table_of_percentiles['BX%']==10,
                                                                                      'Percentile (t)'].values[0])
            layout_graph.title = analysis
            layout_graph.yaxis.title = 'Time (t)'
            layout_graph.xaxis.title = 'Percentage of equipment failed (%)'
            layout_graph.yaxis.type = 'log'
            
            return go.Figure(data=[go.Scatter(x=table_of_percentiles['BX%'].values, y=table_of_percentiles['Percentile (t)'].values,
                                                   marker=dict(
                                                       color='#d04e09',
                                                       line=dict(
                                                           color='#d04e09',
                                                           width=1.5,
                                                       )))], layout=layout_graph)
            
        elif analysis=='MTBF':
            convergence_criterium = 1e-4
            t_final = 130
            t_range = np.arange(0,t_final)
            
            int_MTBF = np.cumsum(1-di.Weibull().cdf(d._json_file['best_model']['MLE']['params'],t_range))
            
            mtbf = int_MTBF[np.max(np.where(np.diff(np.diff(np.diff(int_MTBF)))>convergence_criterium))]  # converging criteria
            
            layout_graph.title = 'MTBF: {0:0.2f}'.format(mtbf)
            layout_graph.yaxis.title = 'cumsum(Reliability)'
            layout_graph.xaxis.title = 'Time (t)'
            
            return go.Figure(data=[go.Scatter(x=t_range, y=int_MTBF,
                                                   marker=dict(
                                                       color='#d04e09',
                                                       line=dict(
                                                           color='#d04e09',
                                                           width=1.5,
                                                       )))], layout=layout_graph)
            
        elif analysis=='ProbPlot':
            t_final = 130
            t_range = np.arange(0,t_final)
            
            ppf_y = di.Weibull().ppf(d._json_file['best_model']['MLE']['params'],t_range)
            
            layout_graph.yaxis.title = 'Percentile'
            layout_graph.xaxis.title = ''
            
            return go.Figure(data=[go.Scatter(x=t_range, y=ppf_y,
                                                   marker=dict(
                                                       color='#d04e09',
                                                       line=dict(
                                                           color='#d04e09',
                                                           width=1.5,
                                                       )))], layout=layout_graph)
            
        else:
            raise NotImplementedError('Type of analysis defined is not implemented yet.')

In [3]:
d = DisplayCalc()

Selected file: ['C:/Users/sergiojesus/Desktop/Diogo2017/EQS_LOCAL/Projects/_NANO/reliability_analytics/RDA_app/data/results/ttf/ttf_symptomNone_client5.json']
JSON loaded:
{'symptom': None, 'ttf': [505, 5, 26, 19, 223, 22, 21, 46, 129, 57, 5, 58, 5, 36, 12, 31, 3, 100, 38, 34, 11, 3, 124, 10, 117, 1, 25, 65, 81, 90], 'client_id': 5, 'client_name': 'MANOLO'}


KeyError: 'best_model'

KeyError: 'best_model'

In [4]:
layout_comp = go.Layout(
            title='Total Components by Component type',
            # xaxis=dict(
            #     title='Equipment',
            # ),
            xaxis=dict(tickangle=40),
            yaxis=dict(title='Count of SAP numbers',)
        )

In [5]:
layout_comp.yaxis.title = 'Time'
layout_comp.xaxis.title = 'Percentage of equipment failed'

In [6]:
d._json_file['best_model']['law']

AttributeError: 'DisplayCalc' object has no attribute '_json_file'

In [7]:
d._json_file['best_model']['LSQ']['params']

AttributeError: 'DisplayCalc' object has no attribute '_json_file'

In [8]:
d._json_file

AttributeError: 'DisplayCalc' object has no attribute '_json_file'

In [9]:
convergence_criterium = 1e-4
int_MTBF = np.cumsum(1-di.Weibull().cdf(d._json_file['best_model']['MLE']['params'],np.arange(0,130)))

AttributeError: 'DisplayCalc' object has no attribute '_json_file'

In [10]:
plt.plot(int_MTBF)

NameError: name 'int_MTBF' is not defined

In [12]:
plt.plot(1-di.Weibull().cdf(d._json_file['best_model']['MLE']['params'],np.arange(0,130)))

AttributeError: 'DisplayCalc' object has no attribute '_json_file'

In [13]:
convergence_criterium

0.0001

In [14]:
int_MTBF[np.argmax(np.diff(np.diff(int_MTBF)))]

NameError: name 'int_MTBF' is not defined

In [15]:
int_MTBF[np.max(np.where(np.diff(np.diff(np.diff(int_MTBF)))>convergence_criterium))]  # converging criteria

NameError: name 'int_MTBF' is not defined

In [16]:
plt.plot(np.diff(np.diff(np.diff(int_MTBF))))

NameError: name 'int_MTBF' is not defined

In [17]:
plt.plot(np.diff(np.diff(np.cumsum(1-di.Weibull().cdf(d._json_file['best_model']['MLE']['params'],np.arange(0,1000,1))))))

AttributeError: 'DisplayCalc' object has no attribute '_json_file'

### Data statistical summary:

In [18]:
# perc = np.linspace(0,1-1e-3,10)*100

perc = np.array([0.01, .1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                  20, 30, 40, 50, 60, 70, 80, 85, 90, 95,
#                 99,
#                 99.9,
                  99.99
                ])

In [ ]:
percentile = di.Weibull().ppf(d._json_file['best_model']['MLE']['params'],perc/100)

table_of_percentiles = pd.DataFrame({'BX%':perc,'Percentile (t)':percentile})
table_of_percentiles

In [ ]:
table_of_percentiles.loc[table_of_percentiles['BX%']==10,'Percentile (t)'].values[0]

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(perc, percentile)
ax.set_xlabel('Percentage of equipment failed')
ax.set_ylabel('Time')

In [ ]:
plt.plot(perc, percentile)
plt.xlabel('Percentage of equipment failed')
plt.ylabel('Time')